In [39]:
%pip install datasets datasets[vision] transformers tqdm pandas torch transformers[torch] torchvision


zsh:1: no matches found: datasets[vision]
Note: you may need to restart the kernel to use updated packages.


In [40]:
from datasets import load_dataset, load_metric
from transformers import AutoImageProcessor
from torchvision.transforms import CenterCrop, Compose, Normalize, Resize, ToTensor, RandomRotation, RandomHorizontalFlip, RandomVerticalFlip
import numpy as np
import torch


In [41]:
model_checkpoint = "microsoft/resnet-50"
batch_size = 32

metric = load_metric("accuracy")

# dataset = load_dataset("imagefolder", data_files={ "train": "datasets/training", "test": "datasets/test" })
dataset = load_dataset("imagefolder", data_dir="./datasets/training")

print(dataset['train'][0])

# dataset.features = Features({
#     'image': Image(),
#     'label': ClassLabel(num_classes=42, names=metadata_df.columns[1:].tolist())
# })

# def process_dicts(input_list):
#     result = []
#     for item in input_list:
#         result.append({
#             'image': item['image'],
#             'label': [key for key, value in item.items() if key != 'image' and value == 1]
#         })

#     return result

# dataset['train'].map(lambda x: print(x), batched=True)

# dataset['train'] = dataset['train'].map(process_dicts, batched=True)

# for i in tqdm.tqdm(range(len(dataset['train']))):
#     dataset['train'][i] = {
#         'image': dataset['train'][i]['image'],
#         'label': [key for key, value in dataset['train'][i].items() if key != 'image' and value == 1]
#     }

# print(dataset['train'][0])
# dataset['train'][0]['image']





Resolving data files: 100%|██████████| 1921/1921 [00:00<00:00, 80234.79it/s]

{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2144x1424 at 0x181030210>, 'Disease_Risk': 1, 'DR': 1, 'ARMD': 0, 'MH': 0, 'DN': 0, 'MYA': 0, 'BRVO': 0, 'TSLN': 0, 'ERM': 0, 'LS': 0, 'MS': 0, 'CSR': 0, 'ODC': 0, 'CRVO': 0, 'TV': 0, 'AH': 0, 'ODP': 0, 'ODE': 0, 'ST': 0, 'AION': 0, 'PT': 0, 'RT': 0, 'RS': 0, 'CRS': 0, 'EDN': 0, 'RPEC': 0, 'MHL': 0, 'RP': 0, 'CWS': 0, 'CB': 0, 'ODPM': 0, 'PRH': 0, 'MNF': 0, 'HR': 0, 'CRAO': 0, 'TD': 0, 'CME': 0, 'PTCR': 0, 'CF': 0, 'VH': 0, 'MCA': 0, 'VS': 0, 'BRAO': 0, 'PLQ': 0, 'HPED': 0, 'CL': 0}


In [42]:
labels = ["Disease_Risk","DR","ARMD","MH","DN","MYA","BRVO","TSLN","ERM","LS","MS","CSR","ODC","CRVO","TV","AH","ODP","ODE","ST","AION","PT","RT","RS","CRS","EDN","RPEC","MHL","RP","CWS","CB","ODPM","PRH","MNF","HR","CRAO","TD","CME","PTCR","CF","VH","MCA","VS","BRAO","PLQ","HPED","CL"]
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = i
    id2label[i] = label


In [43]:
image_processor  = AutoImageProcessor.from_pretrained(model_checkpoint)

normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
if "shortest_edge" in image_processor.size:
    size = image_processor.size["shortest_edge"]
    max_size = image_processor.size.get("longest_edge")

train_transforms = Compose([
    Resize(size),
    CenterCrop((size, size)),
    RandomRotation(360),
    RandomHorizontalFlip(),
    RandomVerticalFlip(),
    ToTensor(),
    normalize
])
val_transforms = Compose([Resize(size), CenterCrop((size, size)), ToTensor(), normalize])

def preprocess_train(example_batch):
    """Apply train_transforms across a batch."""
    example_batch["pixel_values"] = [
        train_transforms(image.convert("RGB")) for image in example_batch["image"]
    ]
    return example_batch

def preprocess_val(example_batch):
    """Apply val_transforms across a batch."""
    example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
    return example_batch


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


In [44]:
splits = dataset["train"].train_test_split(test_size=0.2)
train_ds = splits['train']
val_ds = splits['test']

train_ds.set_transform(preprocess_train)
val_ds.set_transform(preprocess_val)

print(train_ds[0])
print(train_ds[1])
print(train_ds[2])
# print(val_ds)


{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2144x1424 at 0x18644A2D0>, 'Disease_Risk': 1, 'DR': 1, 'ARMD': 0, 'MH': 0, 'DN': 0, 'MYA': 0, 'BRVO': 0, 'TSLN': 0, 'ERM': 0, 'LS': 0, 'MS': 0, 'CSR': 0, 'ODC': 0, 'CRVO': 0, 'TV': 0, 'AH': 0, 'ODP': 0, 'ODE': 0, 'ST': 0, 'AION': 0, 'PT': 0, 'RT': 0, 'RS': 0, 'CRS': 0, 'EDN': 0, 'RPEC': 0, 'MHL': 0, 'RP': 0, 'CWS': 0, 'CB': 0, 'ODPM': 0, 'PRH': 0, 'MNF': 0, 'HR': 0, 'CRAO': 0, 'TD': 0, 'CME': 0, 'PTCR': 0, 'CF': 0, 'VH': 0, 'MCA': 0, 'VS': 0, 'BRAO': 0, 'PLQ': 0, 'HPED': 0, 'CL': 0, 'pixel_values': tensor([[[-0.5596]],

        [[-1.2479]],

        [[-1.6476]]])}
{'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=2144x1424 at 0x186448610>, 'Disease_Risk': 1, 'DR': 0, 'ARMD': 0, 'MH': 0, 'DN': 0, 'MYA': 0, 'BRVO': 0, 'TSLN': 0, 'ERM': 0, 'LS': 0, 'MS': 0, 'CSR': 0, 'ODC': 0, 'CRVO': 0, 'TV': 0, 'AH': 0, 'ODP': 1, 'ODE': 0, 'ST': 0, 'AION': 0, 'PT': 0, 'RT': 0, 'RS': 0, 'CRS': 0, 'EDN': 0, 'RPEC': 1, 'MHL': 0, 'R

In [45]:
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

model = AutoModelForImageClassification.from_pretrained(
    model_checkpoint,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes = True, # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
)
model.config.problem_type = "multi_label_classification"


Some weights of ResNetForImageClassification were not initialized from the model checkpoint at microsoft/resnet-50 and are newly initialized because the shapes did not match:
- classifier.1.weight: found shape torch.Size([1000, 2048]) in the checkpoint and torch.Size([46, 2048]) in the model instantiated
- classifier.1.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([46]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [50]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-eye-disease",
    remove_unused_columns=False,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

# the compute_metrics function takes a Named Tuple as input:
# predictions, which are the logits of the model as Numpy arrays,
# and label_ids, which are the ground-truth labels as Numpy arrays.
def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    references = np.argmax(eval_pred.label_ids, axis=1)
    return metric.compute(predictions=predictions, references=references)


In [51]:
import torch

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    expl_labels = torch.tensor([[float(example[label]) for label in labels] for example in examples])
    return {"pixel_values": pixel_values, "labels": expl_labels}


In [52]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)


In [53]:
train_results = trainer.train()
# rest is optional but nice to have
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()


 33%|███▎      | 12/36 [19:15<38:30, 96.26s/it]














                                               

 33%|███▎      | 12/36 [10:00<07:54, 19.76s/it]


{'loss': 0.6802, 'learning_rate': 4.0625000000000005e-05, 'epoch': 0.83}





























                                            


                                               

                                         

 33%|███▎      | 12/36 [12:44<07:54, 19.76s/it]




{'eval_loss': 0.8625258803367615, 'eval_accuracy': 0.1484375, 'eval_runtime': 112.9112, 'eval_samples_per_second': 3.401, 'eval_steps_per_second': 0.106, 'epoch': 1.0}











                                               

 33%|███▎      | 12/36 [16:43<07:54, 19.76s/it]


...

{'loss': 0.6642, 'learning_rate': 9.375000000000001e-06, 'epoch': 2.5}

































                                            


                                               

                                         

 33%|███▎      | 12/36 [27:18<07:54, 19.76s/it]




{'eval_loss': 2.1224300861358643, 'eval_accuracy': 0.23958333333333334, 'eval_runtime': 81.8431, 'eval_samples_per_second': 4.692, 'eval_steps_per_second': 0.147, 'epoch': 3.0}



                                               

100%|██████████| 36/36 [21:12<00:00, 35.36s/it]


{'train_runtime': 1272.8012, 'train_samples_per_second': 3.62, 'train_steps_per_second': 0.028, 'train_loss': 0.669706854555342, 'epoch': 3.0}
***** train metrics *****
  epoch                    =        3.0
  train_loss               =     0.6697
  train_runtime            = 0:21:12.80
  train_samples_per_second =       3.62
  train_steps_per_second   =      0.028
